In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
from sqlalchemy import create_engine

In [8]:
csv = pd.read_csv('combined.csv').dropna(how='any')
weat = pd.read_csv('weathermap.csv').dropna(how='any')
csv = csv[['City','Mapping','Weather']]
data = csv.merge(weat, left_on='Weather', right_on='Weather')
data = data[['City','Mapping','MapWeather']]
data = data.rename(columns={'MapWeather':'Weather'})
data.head()

,City,Mapping,Weather
0,atlanta,Auto-Theft,Partly Cloudy
1,atlanta,Auto-Theft,Partly Cloudy
2,atlanta,Auto-Theft,Partly Cloudy
3,atlanta,Auto-Theft,Partly Cloudy
4,atlanta,Auto-Theft,Partly Cloudy


In [6]:
# check values
vvv = ['', 'Clear', 'Drizzle', 'Drizzle and Humid', 'Flurries', 'Foggy', 'Heavy Rain','Humid', 'Humid and Foggy', 'Humid and Mostly Cloudy', 
'Humid and Overcast', 'Humid and Partly Cloudy', 'Light Rain', 'Light Rain and Humid', 'Light Sleet', 'Light Snow', 
'Light Snow and Windy', 'Mostly Cloudy', 'Overcast', 'Partly Cloudy', 'Possible Drizzle', 'Possible Drizzle and Humid', 'Possible Drizzle and Windy', 
'Possible Flurries', 'Possible Light Rain', 'Possible Light Rain and Humid', 'Possible Light Sleet', 'Possible Light Snow', 'Rain', 
'Rain and Humid', 'Windy and Partly Cloudy']
bbb = ['atlanta','chicago','denver','boston','los_angeles']

for c in bbb:
    for x in vvv:
        ccc = data.loc[data['City'] == c].loc[data['Weather'] == x]
        xx = ccc['Weather'].count()
        #Uncomment to see values
#         print(c, x, xx)

In [11]:
# Step one get string ids, labels, parents, and numeric value counts for just city 'Atlanta', but format it so easily transformable into 
# later in automated system of loops. Single city system used purely for earror proofing.
city_ls = data['City'].unique()
city_ls = [city_ls[0]]

# Get city list, crime list, and weather list
for x in city_ls:
    loop_df = data.loc[data['City'] == x]
mapping_ls = loop_df['Mapping'].unique()
weather_ls = pd.Series(loop_df['Weather'].unique()).dropna().tolist()
print(city_ls, mapping_ls, weather_ls)

['atlanta'] ['Auto-Theft' 'Robbery' 'Violence' 'Burglary' 'Killing'] ['Partly Cloudy', 'Rain', 'Mostly Cloudy', 'Overcast', 'Clear', 'Snow']


In [12]:
# Step 2: Create sunburst id/label/parent system for just atalanta

parents = []
labels = []
ids = []
values = []

# Use for loop and for loops and for loops, to create labels, ids, parents, and values
for x in city_ls:
    parents.append('')
    labels.append(x)
    values.append((loop_df['City'].value_counts())[x])
    for y in weather_ls:
        parents.append(x)
        xx = loop_df['Weather'].value_counts()
        values.append(xx[y])

labels = labels + weather_ls
print(values)

for y in weather_ls:   
    yy = loop_df.loc[loop_df['Weather'] == y]
    for z in mapping_ls:
        parents.append(y)
        #If type of weather/crime did not occur together, erorr occurs and it will default to 0 
        try:
            zz = yy['Mapping'].value_counts()
            values.append(zz[z])
            labels.append(z)
        except:
            labels.append('')
            values.append(0)
            
        
print(values)
print(len(labels))
print(len(parents))

df = pd.DataFrame({})
df["parents"] = parents
df["labels"] = labels
df["values"] = values

# df.to_csv("ATL_sunburst.csv")
df

[141625, 36270, 15642, 27364, 19430, 42811, 108]
[141625, 36270, 15642, 27364, 19430, 42811, 108, 4917, 2491, 2879, 25843, 140, 1992, 938, 1006, 11658, 48, 3845, 1582, 2076, 19783, 78, 2407, 1129, 1339, 14498, 57, 5399, 2744, 3099, 31429, 140, 10, 7, 5, 85, 1]
37
37


,parents,labels,values
0,,atlanta,141625
1,atlanta,Partly Cloudy,36270
2,atlanta,Rain,15642
3,atlanta,Mostly Cloudy,27364
4,atlanta,Overcast,19430
5,atlanta,Clear,42811
6,atlanta,Snow,108
7,Partly Cloudy,Auto-Theft,4917
8,Partly Cloudy,Robbery,2491
9,Partly Cloudy,Violence,2879


In [13]:
# More value checking
data2 = data[['City','Weather','Mapping']]

data2.groupby(['City']).count()
data2.groupby(['City','Weather']).count()
data2.groupby(['City','Weather','Mapping']).count().reset_index()
xx = data2['City'].value_counts()
xx['atlanta']

141625

In [14]:
#Now repeat step 1 and 2 done for atlanta comined for all cities.
city_ls = data['City'].unique()
parents = []
labels = []
ids = []
values = []

for x in city_ls:
    xc = x.capitalize().replace('Los_angeles','Los Angeles')
    parents.append('')
    labels.append(xc)
    ids.append(f'Weather-{xc}')
    loop_df = data.loc[data['City'] == x]
    values.append((loop_df['City'].value_counts())[x])


for x in city_ls:
    xc = x.capitalize().replace('Los_angeles','Los Angeles')
    loop_df = data.loc[data['City'] == x]
    mapping_ls = loop_df['Mapping'].unique()
    weather_ls = pd.Series(loop_df['Weather'].unique()).dropna().tolist()
    for y in weather_ls:
        parents.append(f'Weather-{xc}')
        xx = loop_df['Weather'].value_counts()
        values.append(xx[y])
        ids.append(f'{xc}-{y}')
        labels.append(y)
        
    
#     print(city_ls, mapping_ls, weather_ls)

    for y in weather_ls:
        yy = loop_df.loc[loop_df['Weather'] == y]
        for z in mapping_ls:
            parents.append(f'{xc}-{y}')
            ids.append(f'{y}-{z}')
            try:
                zz = yy['Mapping'].value_counts()
                values.append(zz[z])
                labels.append(z)
            except:
                labels.append('')
                values.append(0)

# print(len(ids),len(labels), len(parents), len(values))

df = pd.DataFrame({})
df["ids"] = ids
df["labels"] = labels
df["parents"] = parents
df["values"] = values

# df.to_csv("sunburstnova.csv")

print('Mission Accomplished')

Mission Accomplished


In [15]:
df.head()

,ids,labels,parents,values
0,Weather-Atlanta,Atlanta,,141625
1,Weather-Boston,Boston,,329762
2,Weather-Denver,Denver,,446283
3,Weather-Chicago,Chicago,,864007
4,Weather-Los Angeles,Los Angeles,,1090415


In [16]:
# Check to make sure it worked
df.tail()

,ids,labels,parents,values
409,Clear-VAL,VAL,Los Angeles-Clear,475
410,Clear-Sexual,Sexual,Los Angeles-Clear,2252
411,Clear-Killing,Killing,Los Angeles-Clear,818
412,Clear-Arson,Arson,Los Angeles-Clear,1142
413,Clear-Kidnapping,Kidnapping,Los Angeles-Clear,1000


In [17]:
engine = create_engine('sqlite:///sunburst.sqlite')
df.to_sql('sunburst', engine, index = False)

In [18]:
test = pd.read_sql_query('Select * from sunburst',engine)
test.head()

,ids,labels,parents,values
0,Weather-Atlanta,Atlanta,,141625
1,Weather-Boston,Boston,,329762
2,Weather-Denver,Denver,,446283
3,Weather-Chicago,Chicago,,864007
4,Weather-Los Angeles,Los Angeles,,1090415
